# **Build a Dataset Class for Horse Breeds**

https://www.kaggle.com/datasets/olgabelitskaya/horse-breeds

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("olgabelitskaya/horse-breeds")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'horse-breeds' dataset.
Path to dataset files: /kaggle/input/horse-breeds


### Split the data into train val, and test set (starified)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, glob
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torch.nn.functional as F


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [4]:
all_images = []
for ext in ('/**/*.jpg', '/**/*.jpeg', '/**/*.JPG', '/**/*.png'):
    all_images.extend(glob.glob(path + ext, recursive=True))

data = []
for img_path in all_images:
    filename = os.path.basename(img_path)
    breed = filename.split('_')[0] 
    data.append((img_path, breed))
df = pd.DataFrame(data, columns=['filepath', 'label'])

label_to_idx = {label: i for i, label in enumerate(df['label'].unique())}
df['label_idx'] = df['label'].map(label_to_idx)

train_df, temp_df = train_test_split(
    df, test_size=0.20, stratify=df['label_idx'], random_state=42
)

val_df, test_df = train_test_split(
    temp_df, test_size=0.50, stratify=temp_df['label_idx'], random_state=42
)

print(f"Total classes found: {len(label_to_idx)}")
print(df['label'].value_counts().head()) 

Total classes found: 7
label
01    123
06    122
07    120
03    107
02    105
Name: count, dtype: int64


In [ ]:
from torchvision import models

def get_transfer_learned_model(num_classes):
    
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    
    for param in model.parameters():
        param.requires_grad = False
        
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, num_classes)
    )
    
    return model

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tl_model = get_transfer_learned_model(num_classes=len(label_to_idx)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer_tl = optim.Adam(tl_model.parameters(), lr=0.001)

In [5]:
label_to_idx = {label: i for i, label in enumerate(df['label'].unique())}
df['label_idx'] = df['label'].map(label_to_idx)

In [6]:
train_df, temp_df = train_test_split(
    df, test_size=0.20, stratify=df['label_idx'], random_state=42
)

val_df, test_df = train_test_split(
    temp_df, test_size=0.50, stratify=temp_df['label_idx'], random_state=42
)
print(f"Total images: {len(df)}")
print(f"Train size: {len(train_df)} | Val size: {len(val_df)} | Test size: {len(test_df)}")

In [7]:
print(df['label'].value_counts()) 

### Dataset Class

In [ ]:
class HorseBreedDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['filepath']
        label = self.dataframe.iloc[idx]['label_idx']
        
        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
            
        return image, torch.tensor(label, dtype=torch.long)

### Transforms

In [ ]:
def calculate_stats(dataset):
    
    loader = DataLoader(dataset, batch_size=64, shuffle=False)
    
    mean = 0.0
    std = 0.0
    total_images = 0
    
    for images, _ in loader:
        
        batch_samples = images.size(0) 
        images = images.view(batch_samples, images.size(1), -1)
        

        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images += batch_samples

    
    mean /= total_images
    std /= total_images
    
    return mean, std


temp_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

temp_dataset = HorseBreedDataset(df, transform=temp_transforms)
mean, std = calculate_stats(temp_dataset)

print(f"Calculated Mean: {mean}")
print(f"Calculated Std: {std}")

In [ ]:
my_mean = [0.5216, 0.5143, 0.4437]
my_std = [0.2361, 0.2397, 0.2472]

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(my_mean, my_std)
])

val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(my_mean, my_std)
])

### Create Dataloader objects

In [ ]:
train_set = HorseBreedDataset(train_df, transform=train_transforms)
val_set = HorseBreedDataset(val_df, transform=val_test_transforms)
test_set = HorseBreedDataset(test_df, transform=val_test_transforms)

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

#### Display some images

In [ ]:
def show_images(dataset, num_images=5):
    plt.figure(figsize=(15, 5))
    for i in range(num_images):
        img, label = dataset[i]
        img = img.permute(1, 2, 0).numpy()
        img = np.clip(img * my_std + my_mean, 0, 1) 
        
        plt.subplot(1, num_images, i+1)
        plt.imshow(img)
        plt.title(list(label_to_idx.keys())[label])
        plt.axis('off')
    plt.show()

show_images(train_set)

### Define Model 

In [ ]:
class NatureCNN_Horse(nn.Module):
    def __init__(self, num_classes):
        super(NatureCNN_Horse, self).__init__()
        
        # Block 1: Input (3, 224, 224) -> Output (16, 112, 112)
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        # Block 2: Input (16, 112, 112) -> Output (32, 56, 56)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        # Block 3: Input (32, 56, 56) -> Output (64, 28, 28)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        self.adaptive_pool = nn.AdaptiveAvgPool2d((7, 7))
        
        self.flatten = nn.Flatten()
        
        self.dropout = nn.Dropout(0.3)
    
        self.fc1 = nn.Linear(64 * 7 * 7, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        
        x = self.adaptive_pool(x)
        x = self.flatten(x)
        
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x
    


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NatureCNN_Horse(num_classes=len(label_to_idx)).to(device)

### define Loss and Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#### Build one_epoch_training function loop 

In [ ]:
def training_loop(model, train_loader, val_loader, loss_function, optimizer, num_epochs, device):

    model.to(device)
    
    train_losses = []
    val_losses = []
    val_accuracies = []
    
    best_val_loss = float('inf')
    
    print("--- Training Started ---")
    
    for epoch in range(num_epochs):
    
        model.train()
        running_loss = 0.0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * images.size(0)
            
        epoch_loss = running_loss / len(train_loader.dataset)
        train_losses.append(epoch_loss)
        
       
        model.eval()
        running_val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                
                outputs = model(images)
                val_loss = loss_function(outputs, labels)
                
                running_val_loss += val_loss.item() * images.size(0)
                
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
        epoch_val_loss = running_val_loss / len(val_loader.dataset)
        val_losses.append(epoch_val_loss)
        
        epoch_accuracy = 100.0 * correct / total
        val_accuracies.append(epoch_accuracy)
        
      
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            save_msg = "(Best Model Saved!)"
        else:
            save_msg = ""

        
        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {epoch_loss:.4f}, "
              f"Val Loss: {epoch_val_loss:.4f}, "
              f"Val Acc: {epoch_accuracy:.2f}% {save_msg}")
        
    print("--- Finished Training ---")
    
    
    metrics = [train_losses, val_losses, val_accuracies]
    

    model.load_state_dict(torch.load('best_model.pth'))
    
    return model, metrics

#### Build one_epoch_validation function loop 

In [ ]:
class HelperUtils:

    def plot_training_metrics(self, metrics):
        train_losses, val_losses, val_accuracies = metrics
        epochs = range(1, len(train_losses) + 1)

        plt.figure(figsize=(12, 5))

    
        plt.subplot(1, 2, 1)
        plt.plot(epochs, train_losses, label='Training Loss', color='blue', marker='o')
        plt.plot(epochs, val_losses, label='Validation Loss', color='red', marker='o')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.grid(True)
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(epochs, val_accuracies, label='Validation Accuracy', color='green', marker='s')
        plt.title('Validation Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy (%)')
        plt.grid(True)
        plt.legend()

        plt.tight_layout()
        plt.show()


helper_utils = HelperUtils()

In [ ]:
from torchvision import models

def get_transfer_learned_model(num_classes):
    
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    
    for param in model.parameters():
        param.requires_grad = False
        
  
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, num_classes)
    )
    
    return model


tl_model = get_transfer_learned_model(num_classes=len(label_to_idx)).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_tl = optim.Adam(tl_model.parameters(), lr=0.001)


trained_tl_model, training_metrics_tl = training_loop(
    model=tl_model,         
    train_loader=train_loader, 
    val_loader=val_loader, 
    loss_function=criterion, 
    optimizer=optimizer_tl, 
    num_epochs=15, 
    device=device
)

print("\n--- Transfer Learning Training Plots ---\n")
helper_utils.plot_training_metrics(training_metrics_tl)

### Combine all to train the model
it should Save the best model and track train and val loss and accuracy


### test the model on test set

### show some predictions with the images

### Analyze the results
Is the model overfitting/underfitting?
Plot the training and validation loss/accuracy curves

### Load the model